## 전처리 코드 전문

### XML 파일 Formatting

In [ ]:
import xml.etree.ElementTree as ET

def pretty_print_xml(input_file, output_file):
    """
    XML 파일을 읽어 들여쓰기를 적용한 뒤, 
    태그와 구조가 보이도록 새 파일로 저장합니다.
    (Python 3.9 이상 권장)
    """
    try:
        # 1. XML 파일 파싱
        tree = ET.parse(input_file)
        root = tree.getroot()

        # 2. XML 들여쓰기 적용 (핵심!)
        # Python 3.9부터 사용 가능
        ET.indent(root, space="  ")  # 스페이스 2칸으로 들여쓰기

        # 3. 예쁘게 정렬된 XML을 새 파일로 저장
        # xml_declaration=True는 '<?xml version="1.0" ...?>' 선언부를 추가합니다.
        tree.write(output_file, encoding='utf-8', xml_declaration=True)
        
        print(f"'{input_file}'을 파싱하여 '{output_file}'에 예쁘게 저장했습니다.")

    except ET.ParseError as e:
        print(f"XML 파싱 오류: {e}")
    except FileNotFoundError:
        print(f"오류: '{input_file}' 파일을 찾을 수 없습니다.")
    except AttributeError as e:
        print(f"오류: {e}")
        print("참고: ET.indent() 함수는 Python 3.9 이상에서만 동작합니다.")
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")

# --- 사용 예시 ---
# 'input.xml' 파일을 읽어서 'output_pretty.xml' 파일로 저장
input_xml_file = "../raw/EN_ipc_scheme_20250101.xml"       # 원본 XML 파일 경로
output_xml_file = "../raw/output_pretty.xml"  # 저장할 파일 경로

pretty_print_xml(input_xml_file, output_xml_file)

'../EN_ipc_scheme_20250101.xml'을 파싱하여 'output_pretty.xml'에 예쁘게 저장했습니다.


### IPC코드 Json화 (WIPO 표시 형식)

In [ ]:
import xml.etree.ElementTree as ET
import json
from tqdm import tqdm
import re  # 정규 표현식을 위해 import

# --- 1. IPC 심볼 포맷팅 함수 (⭐ 새로 추가됨) ---

def format_ipc_symbol(symbol):
    """
    'A01B0001040000' 같은 긴 심볼을 'A01B1/04' 형식으로 변환합니다.
    짧은 심볼('A', 'A01B')은 그대로 반환합니다.
    """
    
    # 1. 정규표현식으로 Subclass 부분과 숫자 부분을 분리
    #    (예: "A01B"와 "0001040000"으로 분리)
    #    IPC Subclass는 "H05K"처럼 [A-H][0-9][0-9][A-Z] 형식을 따릅니다.
    match = re.match(r"([A-H][0-9]{2}[A-Z])([0-9]+)$", symbol)
    
    # 2. 매치되지 않으면 (e.g., "A", "A01", "A01B") 원본 심볼 반환
    if not match:
        return symbol

    prefix = match.group(1)       # "A01B"
    numeric_part = match.group(2) # "0001040000"

    # 3. 숫자 부분이 6자리(메인+서브) 이상인지 확인
    if len(numeric_part) < 6:
        return symbol # 포맷팅 불가, 원본 반환
        
    # 4. 메인 그룹 (앞 4자리)
    # "0001" -> 1 -> "1"
    main_group = str(int(numeric_part[0:4]))
    
    # 5. 서브 그룹 (다음 2자리)
    # "04"
    sub_group = numeric_part[4:6]
    
    # 6. 조합
    # 서브 그룹이 "00"이면 "A01B1/00"
    if sub_group == "00":
        return f"{prefix}{main_group}/00"
    # 서브 그룹이 "00"이 아니면 "A01B1/04"
    else:
        return f"{prefix}{main_group}/{sub_group}"

# --- 2. XML 파싱을 위한 헬퍼 함수 (수정됨) ---

def extract_title(element, namespaces):
    title_parts = []
    text_body = element.find('ns0:textBody', namespaces)
    if text_body is None: return ""
    title = text_body.find('ns0:title', namespaces)
    if title is None: return ""
    for title_part in title.findall('ns0:titlePart', namespaces):
        text_element = title_part.find('ns0:text', namespaces)
        if text_element is not None and text_element.text:
            title_parts.append(text_element.text.strip())
    return " ".join(title_parts)

def parse_recursive(element, namespaces, parent_symbol, ancestor_path):
    """
    <ns0:ipcEntry> 요소를 재귀적으로 파싱
    (⭐ 심볼 포맷팅 기능 적용됨)
    """
    processed_list = []
    
    # 1. 원본 심볼 가져오기
    raw_symbol = element.get('symbol')
    
    # 2. (⭐ 수정) 심볼 포맷팅
    formatted_symbol = format_ipc_symbol(raw_symbol)
    
    kind = element.get('kind')
    
    # 3. (⭐ 수정) 포맷팅된 심볼을 path와 parent에 사용
    current_path = ancestor_path + [formatted_symbol]
    title = extract_title(element, namespaces)
    
    current_item = {
        "ipc_code": formatted_symbol, # 포맷팅된 코드로 저장
        "title_en": title,
        "kind": kind,
        "parent": parent_symbol,
        "path": current_path
    }
    processed_list.append(current_item)
    
    for child_element in element.findall('ns0:ipcEntry', namespaces):
        processed_list.extend(
            parse_recursive(child_element, namespaces, 
                            parent_symbol=formatted_symbol, # 자식에게 포맷팅된 코드를 물려줌
                            ancestor_path=current_path)
        )
        
    return processed_list

# --- 3. 중복 필터링 헬퍼 함수 (동일) ---

def filter_ipc_data(data):
    filtered_items = {}
    print("\n중복 필터링 시작...")
    for item in tqdm(data, desc="필터링 중"):
        code = item['ipc_code']
        kind = item['kind']
        if code not in filtered_items:
            filtered_items[code] = item
            continue
        existing_item = filtered_items[code]
        existing_kind = existing_item['kind']
        if existing_kind == 't' and kind == 'c':
            filtered_items[code] = item
    return list(filtered_items.values())

# --- 4. 메인 실행 함수 (동일) ---

def process_xml_to_filtered_json(input_xml_file, output_json_file):
    try:
        namespaces = {'ns0': 'http://www.wipo.int/classifications/ipc/masterfiles'}
        print(f"'{input_xml_file}' 파일 파싱 시작...")
        tree = ET.parse(input_xml_file)
        root = tree.getroot()
        all_ipc_data = []
        top_level_entries = root.findall('ns0:ipcEntry', namespaces)
        if not top_level_entries:
            print("오류: 최상위 <ns0:ipcEntry> 태그를 찾을 수 없습니다.")
            return
        print("XML 구조 분석 중...")
        for entry in tqdm(top_level_entries, desc="파싱 중 (최상위)"):
            all_ipc_data.extend(parse_recursive(entry, namespaces, 
                                                parent_symbol=None, 
                                                ancestor_path=[]))
        print(f"\n파싱 완료. (필터링 전 총 {len(all_ipc_data)}개 항목)")
        
        filtered_data = filter_ipc_data(all_ipc_data)
        
        print(f"\n필터링 완료. (최종 {len(filtered_data)}개 항목)")
        
        print(f"'{output_json_file}' 파일 저장 중...")
        with open(output_json_file, 'w', encoding='utf-8') as f:
            json.dump(filtered_data, f, ensure_ascii=False, indent=2)

        print(f"\n--- 모든 작업 완료 ---")
        print(f"입력: {input_xml_file}")
        print(f"출력: {output_json_file}")
    except ET.ParseError as e:
        print(f"XML 파싱 오류: {e}")
    except FileNotFoundError:
        print(f"오류: '{input_xml_file}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")

# --- 사용 예시 ---

# (중요) 실제 파일 경로로 수정하세요.
input_file = "../raw/EN_ipc_scheme_20250101.xml"  # 👈 여기에 실제 원본 XML 파일 경로
output_file = "../raw/json/ipc_filtered_data_formatted.json"   # 👈 여기에 저장할 JSON 파일 경로

process_xml_to_filtered_json(input_file, output_file)

'../EN_ipc_scheme_20250101.xml' 파일 파싱 시작...
XML 구조 분석 중...


파싱 중 (최상위): 100%|██████████| 8/8 [00:01<00:00,  5.71it/s]



파싱 완료. (필터링 전 총 81818개 항목)

중복 필터링 시작...


필터링 중: 100%|██████████| 81818/81818 [00:00<00:00, 2086517.69it/s]



필터링 완료. (최종 69337개 항목)
'ipc_filtered_data_formatted.json' 파일 저장 중...

--- 모든 작업 완료 ---
입력: ../EN_ipc_scheme_20250101.xml
출력: ipc_filtered_data_formatted.json


### JSON계층 구조에서 종속형 코드 처리

In [6]:
import json
from tqdm import tqdm  # tqdm 라이브러리 import

def process_ipc_json(input_file_path, output_file_path):
    """
    JSON 파일을 읽어, path 기반으로 title_en을 재조합한 뒤 
    새 JSON 파일로 저장합니다. (tqdm 진행률 표시)

    Args:
        input_file_path (str): 원본 JSON 파일 경로
        output_file_path (str): 저장할 JSON 파일 경로
    """
    try:
        # 1. 원본 JSON 파일 읽기 (tqdm 적용이 크게 의미 없음)
        print(f"'{input_file_path}' 파일 읽는 중...")
        with open(input_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"파일 읽기 완료 (총 {len(data)}개 항목).")


        # 2. 'ipc_code'와 원본 'title_en' 조회용 맵 생성 (tqdm 적용이 크게 의미 없음)
        print("조회용 맵 생성 중...")
        title_lookup = {item['ipc_code']: item['title_en'] for item in data}
        print("맵 생성 완료.")


        processed_data = []

        # 3. 각 항목을 순회하며 'title_en'을 새로 조합 (tqdm 적용)
        print("데이터 처리 시작...")
        # ⭐ tqdm으로 data 리스트를 감싸서 진행률 표시
        for item in tqdm(data, desc="데이터 처리 중"):
            # path의 각 코드에 해당하는 title을 조회
            path_titles = [title_lookup[code] for code in item['path']]
            
            # title들을 공백으로 합치기
            new_title = " ".join(path_titles)
            
            # 원본 데이터를 복사하고 title_en만 교체
            new_item = item.copy()
            new_item['title_en'] = new_title
            
            processed_data.append(new_item)

        # 4. 처리된 데이터를 새 JSON 파일로 저장 (tqdm 적용이 크게 의미 없음)
        print(f"\n'{output_file_path}' 파일 저장 중...")
        with open(output_file_path, 'w', encoding='utf-8') as f:
            json.dump(processed_data, f, ensure_ascii=False, indent=2)

        print(f"파일 처리가 완료되었습니다. (저장 위치: {output_file_path})")

    except FileNotFoundError:
        print(f"오류: '{input_file_path}' 파일을 찾을 수 없습니다.")
    except KeyError as e:
        print(f"오류: 'path'에 잘못된 ipc_code가 포함되어 있습니다. ({e})")
    except Exception as e:
        print(f"알 수 없는 오류가 발생했습니다: {e}")

# --- 사용 예시 ---
# "original_data.json" 파일을 읽어서 "processed_data.json" 파일로 저장
input_file = "../raw/json/ipc_filtered_data_formatted.json"
output_file = "processed_data.json"
process_ipc_json(input_file, output_file)

'../raw/json/ipc_filtered_data_formatted.json' 파일 읽는 중...
파일 읽기 완료 (총 69337개 항목).
조회용 맵 생성 중...
맵 생성 완료.
데이터 처리 시작...


데이터 처리 중: 100%|██████████| 69337/69337 [00:00<00:00, 306462.94it/s]



'processed_data.json' 파일 저장 중...
파일 처리가 완료되었습니다. (저장 위치: processed_data.json)


## 완성된 JSON파일에서 title_en 길이 검사

In [7]:
import json

def analyze_title_lengths(input_file_path):
    """
    JSON 파일을 읽어 'title_en' 필드의 
    최대 길이와 평균 길이를 계산합니다.

    Args:
        input_file_path (str): 분석할 JSON 파일 경로

    Returns:
        tuple: (max_length, avg_length)
               (오류 발생 시 None, None)
    """
    lengths = []
    
    try:
        # 1. 파일 읽기
        with open(input_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        if not data:
            print("오류: 파일이 비어있습니다.")
            return 0, 0.0

        # 2. 'title_en' 키의 길이 계산
        for item in data:
            # .get()을 사용하면 'title_en' 키가 없어도 오류가 나지 않음
            title = item.get('title_en')
            if title is not None:
                lengths.append(len(title))
            else:
                # title_en 키가 없는 항목이 있다면 경고
                print(f"Warning: 'title_en' 키가 없는 항목 발견 (ipc_code: {item.get('ipc_code')})")

        # 3. 통계 계산
        if not lengths:
            print("오류: 'title_en' 필드를 가진 데이터를 찾을 수 없습니다.")
            return 0, 0.0
            
        max_length = max(lengths)
        avg_length = sum(lengths) / len(lengths)
        
        return max_length, avg_length

    except FileNotFoundError:
        print(f"오류: '{input_file_path}' 파일을 찾을 수 없습니다.")
        return None, None
    except json.JSONDecodeError:
        print(f"오류: '{input_file_path}' 파일이 올바른 JSON 형식이 아닙니다.")
        return None, None
    except Exception as e:
        print(f"알 수 없는 오류 발생: {e}")
        return None, None

# 2. 함수 실행
input_file = "processed_data.json"
max_len, avg_len = analyze_title_lengths(input_file)

# 3. 결과 출력
if max_len is not None:
    print(f"\n--- 분석 결과 ({input_file}) ---")
    print(f"최대 길이: {max_len}")
    print(f"평균 길이: {avg_len:.2f}") # 소수점 2자리까지 표시


--- 분석 결과 (processed_data.json) ---
최대 길이: 1518
평균 길이: 336.94
